In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GRU, Bidirectional, TimeDistributed, BatchNormalization, Embedding

from numpy import array
from keras.models import load_model
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import os

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.filterwarnings('ignore')

/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
scl = preprocessing.StandardScaler()

#https://www.kaggle.com/mikhailhushchyn/dbscan-benchmark
#https://www.kaggle.com/mikhailhushchyn/hough-transform
def norm_points(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2 + y**2 + z**2)
    df['x2'] = x/r
    df['y2'] = y/r
    df['z2'] = z / np.sqrt(x**2 + y**2)
    return df


In [3]:
def append_to_csv(batch, csv_file):
    props = dict(encoding='utf-8', index=False)
    if not os.path.exists(csv_file):
        batch.to_csv(csv_file, **props)
    else:
        batch.to_csv(csv_file, mode='a', header=False, **props)

def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

In [4]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [5]:
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm

class Clusterer1(object):
    def __init__(self,rz_scales=[0.65, 0.965, 1.428]):                        
        self.rz_scales=rz_scales
    
    def _eliminate_outliers(self,labels,M):
        norms=np.zeros((len(labels)),np.float32)
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if cluster == 0:
                continue
            index = np.argwhere(self.clusters==cluster)
            x = M[index]
            norms[i] = self._test_quadric(x)
        threshold = np.percentile(norms,90)*5.3
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if norms[i] > threshold:
                self.clusters[self.clusters==cluster]=0            
    def _test_quadric(self,x):
        if len(x.shape)==3:
            x = np.reshape(x,(x.shape[0],3))
        Z = np.zeros((x.shape[0],10), np.float32)
        Z[:,0] = x[:,0]**2
        Z[:,1] = 2*x[:,0]*x[:,1]
        Z[:,2] = 2*x[:,0]*x[:,2]
        Z[:,3] = 2*x[:,0]
        Z[:,4] = x[:,1]**2
        Z[:,5] = 2*x[:,1]*x[:,2]
        Z[:,6] = 2*x[:,1]
        Z[:,7] = x[:,2]**2
        Z[:,8] = 2*x[:,2]
        Z[:,9] = 1
        v, s, t = np.linalg.svd(Z,full_matrices=False)        
        smallest_index = np.argmin(np.array(s))
        T = np.array(t)
        T = T[smallest_index,:]        
        norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
        return norm

    def _preprocess(self, hits):
        
        x = hits.x.values
        y = hits.y.values
        z = hits.z.values

        r = np.sqrt(x**2 + y**2 + z**2)
        hits['x2'] = x/r
        hits['y2'] = y/r

        r = np.sqrt(x**2 + y**2)
        hits['z2'] = z/r

        ss = StandardScaler()
        X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
        for i, rz_scale in enumerate(self.rz_scales):
            X[:,i] = X[:,i] * rz_scale

        
        return X
    
    def predict(self, hits):        
        volumes = np.unique(hits['volume_id'].values)
        X = self._preprocess(hits)
        self.clusters = np.zeros((len(X),1),np.int32)
        max_len = 1
        cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
                             metric='braycurtis',cluster_selection_method='leaf',algorithm='best', leaf_size=100)
        self.clusters = cl.fit_predict(X)+1
        labels = np.unique(self.clusters)
        n_labels = 0
        while n_labels < len(labels):
            n_labels = len(labels)
            self._eliminate_outliers(labels,X)
            max_len = np.max(self.clusters)
            self.clusters[self.clusters==0] = cl.fit_predict(X[self.clusters==0])+max_len
            labels = np.unique(self.clusters)
        return self.clusters

In [5]:
hits = pd.read_csv('../cache/hits_6488.csv')
hits = hits[hits.nhits >= 9]

hits['new_pid'] = hits.particle_id.astype('str') + '_' + hits.event_id.astype('str')
new_pid_list = list(set(hits.new_pid.values))

new_pid_count = list(range(len(new_pid_list)))
print(len(new_pid_count))

new_pid_dict = dict(zip(new_pid_list, new_pid_count))

hits['nid'] = hits['new_pid'].map(lambda x: new_pid_dict[x])
hits.sort_values(['nid', 'z'], inplace=True)
hits.to_csv('../cache/hits_6488_nid_z_sorted_ge9.csv', index=False)
print(hits.shape)
print(hits.particle_id.nunique())
print(hits.new_pid.nunique())

10149
(121814, 26)
10149
10149


In [5]:
hits = pd.read_csv('../cache/hits_6488_nid_z_sorted_ge9.csv')

In [6]:
model = Sequential()
# model.add(BatchNormalization(input_shape=(1,2)))
model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(1,3)))
# model.add(Bidirectional(LSTM(100, return_sequences=True)))
# model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Bidirectional(LSTM(100)))
# model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))

# model.add(Dense(100000, activation='relu'))

# model.add(Dropout(0.1))
model.add(Dense(10149, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [7]:
model = load_model('../cache/checkpoint/checkpoint-6488-copy2--174.hdf5')

In [8]:
str1='../cache/checkpoint/checkpoint-6488-copy2'
str2 = '{epoch:02d}.hdf5'
filepath = str1 + '-' + str2
checkpoint = ModelCheckpoint(filepath, verbose=0, save_best_only=False)
callbacks_list = [checkpoint]

X_train = hits[['x', 'y', 'z']].values

y = hits['nid'].values
print(y)
print(X_train.shape)

model.fit(X_train.reshape(X_train.shape[0], 1, 3), y, batch_size=32, epochs=3000, shuffle=False, verbose=0, 
      callbacks=callbacks_list, initial_epoch=174)

[    0     0     0 ... 10148 10148 10148]
(121814, 3)


In [9]:
model.save('../cache/model-lstm-2-1-copy2.h5')

In [10]:
path_to_train = "../data/train"
dataset_submissions = []
dataset_scores = []

for event_id, hits, cells, particles, truth in load_dataset(path_to_train, skip=11, nevents=1):
    # Track pattern recognition
#     model = Clusterer()
    X_test = hits[['x', 'y', 'z']].values
#     labels = model.predict(hits)
    labels = model.predict(X_test.reshape(X_test.shape[0],1,3))
#     hits['track_id'] = labels
    
    labels = [np.argmax(labels[i]) for i in range(len(labels))]
#     print(labels)
    print(len(list(set(labels))))
    print(len(list(set(particles.particle_id.values))))
    # Prepare submission for an event
    one_submission = create_one_event_submission(event_id, hits, labels)
    dataset_submissions.append(one_submission)

    # Score for the event
    score = score_event(truth, one_submission)
    dataset_scores.append(score)

    print("Score for event %d: %.9f" % (event_id, score))
print('Mean score: %.9f' % (np.mean(dataset_scores)))

2109
11630
Score for event 1011: 0.004964795
Mean score: 0.004964795


In [11]:
hits = pd.read_csv('../cache/hits_6488_nid_z_sorted_ge9.csv')

In [13]:
hits.head(20)

,hit_id,x,y,z,volume_id,layer_id,module_id,particle_id,tx,ty,...,vy,vz,px,py,pz,q,nhits,event_id,new_pid,nid
0,84143,201.71700,243.5090,-2948.500,12,2,109,22527962460979200,201.85000,243.6100,...,0.008188,8.22454,0.344363,0.312641,-4.34823,-1.0,12.0,6488,22527962460979200_6488,0
1,85697,176.55900,209.7920,-2548.500,12,4,109,22527962460979200,176.37000,209.5950,...,0.008188,8.22454,0.344363,0.312641,-4.34823,-1.0,12.0,6488,22527962460979200_6488,0
2,1183,109.93700,119.0230,-1502.000,7,2,69,22527962460979200,109.93500,119.0130,...,0.008188,8.22454,0.344363,0.312641,-4.34823,-1.0,12.0,6488,22527962460979200_6488,0
3,3318,96.49130,102.2540,-1302.000,7,4,69,22527962460979200,96.49550,102.2350,...,0.008188,8.22454,0.344363,0.312641,-4.34823,-1.0,12.0,6488,22527962460979200_6488,0
4,5919,82.60110,85.6770,-1102.000,7,6,69,22527962460979200,82.60960,85.6684,...,0.008188,8.22454,0.344363,0.312641,-4.34823,-1.0,12.0,6488,22527962460979200_6488,0
5,12526,62.67810,63.0994,-822.500,7,10,66,22527962460979200,62.69210,63.1013,...,0.008188,8.22454,0.344363,0.312641,-4.34823,-1.0,12.0,6488,22527962460979200_6488,0
6,12600,62.36540,62.7425,-818.000,7,10,68,22527962460979200,62.36580,62.7424,...,0.008188,8.22454,0.344363,0.312641,-4.34823,-1.0,12.0,6488,22527962460979200_6488,0
7,16463,53.57880,53.2246,-698.000,7,12,68,22527962460979200,53.58030,53.2297,...,0.008188,8.22454,0.344363,0.312641,-4.34823,-1.0,12.0,6488,22527962460979200_6488,0
8,20432,46.58510,45.7382,-602.500,7,14,66,22527962460979200,46.58620,45.7440,...,0.008188,8.22454,0.344363,0.312641,-4.34823,-1.0,12.0,6488,22527962460979200_6488,0
9,20492,46.24730,45.4105,-598.000,7,14,68,22527962460979200,46.25460,45.3938,...,0.008188,8.22454,0.344363,0.312641,-4.34823,-1.0,12.0,6488,22527962460979200_6488,0


In [14]:
hits.particle_id.nunique()

10149

In [ ]:

# Following previous answers, The number of parameters of LSTM, 
# taking input vectors of size m and giving output vectors of size n is:

# 4(nm+n2)
n = 10149
m = 3

4 * (10149 * 3 + 9)

In [ ]:
4 * (10149 * 3 + 9)

In [16]:
# model = Sequential()
# # model.add(BatchNormalization(input_shape=(1,2)))
# model.add(Bidirectional(LSTM(20000, return_sequences=True), input_shape=(1,3)))
# # model.add(Bidirectional(LSTM(100, return_sequences=True)))
# # model.add(Bidirectional(LSTM(100, return_sequences=True)))
# model.add(Bidirectional(LSTM(20000)))
# # model.add(Dropout(0.3))
# model.add(Dense(16384, activation='relu'))

# # model.add(Dense(100000, activation='relu'))

# # model.add(Dropout(0.1))
# model.add(Dense(10149, activation='softmax'))
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [6]:
model = load_model('../cache/checkpoint/checkpoint-6488-20-1-3000.hdf5')

In [24]:
model_100 = load_model('../cache/checkpoint/checkpoint-6488-100-1-3000.hdf5')

In [35]:
model_20 = load_model('../cache/checkpoint/checkpoint-6488-20-3000.hdf5')

In [7]:
path_to_train = "../data/train"
dataset_submissions = []
dataset_scores = []

mylabels1 = {}
for event_id, hits, cells, particles, truth in load_dataset(path_to_train, skip=10, nevents=1):
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')

    hits = hits.dropna()
    truth = truth.dropna()

    # Track pattern recognition
    model1 = Clusterer1()
    labs = model1.predict(hits)
    hits['labels'] = labs
    mylabels1[event_id] = labs
    
    all_hits = pd.DataFrame()
    prev_p_len = 0
    for i, label in tqdm(enumerate(list(set(labs)))):
#         print(i)
#         print(label)
        myhits = hits[hits['labels'] == label]
        X_test = myhits[['x2', 'y2', 'z2']].values

        p = model.predict(X_test.reshape(X_test.shape[0],1,3))    
        p = [np.argmax(p[j]) + 1 + (i * prev_p_len) for j in range(len(p))]
        prev_p_len = len(p)
        if prev_p_len < 100:
            prev_p_len = 100
        num_instances = [p.count(j) for j in p]
        myhits['labels1'] = p
        myhits['num_label_instances'] = num_instances
        all_hits = all_hits.append(myhits)
    all_hits_new = all_hits[all_hits.num_label_instances <= 3]
    all_hits_new1 = all_hits[all_hits.num_label_instances <= 4]
    all_hits_new2 = all_hits[all_hits.num_label_instances <= 5]
    
    print(all_hits_new.shape)
    print(all_hits_new1.shape)
    print(all_hits_new2.shape)
    all_hits.sort_values('hit_id', inplace=True)
    # Prepare submission for an event
    one_submission = create_one_event_submission(event_id, all_hits, all_hits['labels1'].values)
#     dataset_submissions.append(one_submission)

    # Score for the event
    score = score_event(all_hits, one_submission)
    dataset_scores.append(score)

    print("Score for event %d: %.9f" % (event_id, score))
print('Mean score: %.9f' % (np.mean(dataset_scores)))

100%|██████████| 6658/6658 [00:00<00:00, 211463.63it/s]
6657it [21:17,  5.21it/s]


(352, 29)
(552, 29)
(787, 29)
Score for event 1010: 0.254156013
Mean score: 0.254156013


In [25]:
path_to_train = "../data/train"
dataset_submissions = []
dataset_scores = []

mylabels1 = {}
for event_id, hits, cells, particles, truth in load_dataset(path_to_train, skip=10, nevents=1):
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')

    hits = hits.dropna()
    truth = truth.dropna()

    # Track pattern recognition
    model1 = Clusterer1()
    labs = model1.predict(hits)
    hits['labels'] = labs
    mylabels1[event_id] = labs
    
    all_hits = pd.DataFrame()
    prev_p_len = 0
    for i, label in tqdm(enumerate(list(set(labs)))):
#         print(i)
#         print(label)
        myhits = hits[hits['labels'] == label]
        X_test = myhits[['x2', 'y2', 'z2']].values

        p = model.predict(X_test.reshape(X_test.shape[0],1,3))    
        p = [np.argmax(p[j]) + 1 + (i * prev_p_len) for j in range(len(p))]
        prev_p_len = len(p)
        if prev_p_len < 100:
            prev_p_len = 100
        num_instances = [p.count(j) for j in p]
        myhits['labels1'] = p
        myhits['num_label_instances'] = num_instances
        all_hits = all_hits.append(myhits)
    all_hits_new = all_hits[all_hits.num_label_instances <= 3]
    all_hits_new1 = all_hits[all_hits.num_label_instances <= 4]
    all_hits_new2 = all_hits[all_hits.num_label_instances <= 5]
    hit_list = list(set(all_hits_new2.hit_id.values))
    all_hits = all_hits[~all_hits.hit_id.isin(hit_list)]
    
    X_test = all_hits_new2[['x2', 'y2', 'z2']].values

    p = model_100.predict(X_test.reshape(X_test.shape[0],1,3))    
    p = [np.argmax(p[j]) + 1 + (i * 100000) for j in range(len(p))]


    all_hits_new2['labels1'] = p

    all_hits = all_hits.append(all_hits_new2)
    print(all_hits_new.shape)
    print(all_hits_new1.shape)
    print(all_hits_new2.shape)
    all_hits.sort_values('hit_id', inplace=True)
    # Prepare submission for an event
    one_submission = create_one_event_submission(event_id, all_hits, all_hits['labels1'].values)
#     dataset_submissions.append(one_submission)

    # Score for the event
    score = score_event(all_hits, one_submission)
    dataset_scores.append(score)

    print("Score for event %d: %.9f" % (event_id, score))
print('Mean score: %.9f' % (np.mean(dataset_scores)))

100%|██████████| 6658/6658 [00:00<00:00, 162959.26it/s]
6657it [34:31,  3.21it/s]


NameError: name 'all_list_new2' is not defined

In [40]:
hit_list = list(set(all_hits_new2.hit_id.values))
print(all_hits.shape)
all_hits = all_hits[~all_hits.hit_id.isin(hit_list)]
print(all_hits.shape)
X_test = all_hits_new2[['x', 'y', 'z']].values

p = model_100.predict(X_test.reshape(X_test.shape[0],1,3))    
p = [np.argmax(p[j]) + 1 + (i * 2 * 100000) for j in range(len(p))]


all_hits_new2['labels1'] = p

all_hits = all_hits.append(all_hits_new2)
print(all_hits_new.shape)
print(all_hits_new1.shape)
print(all_hits_new2.shape)
print(all_hits.shape)
all_hits.sort_values('hit_id', inplace=True)
# Prepare submission for an event
one_submission = create_one_event_submission(0, all_hits, all_hits['labels1'].values)
#     dataset_submissions.append(one_submission)

# Score for the event
score = score_event(all_hits, one_submission)
dataset_scores.append(score)

print("Score for event %d: %.9f" % (event_id, score))

(86310, 30)
(85523, 30)
(352, 29)
(552, 29)
(787, 30)
(86310, 30)
Score for event 1010: 0.253828278


In [9]:
all_hits_new2.sort_values('particle_id',inplace=True)

In [10]:
all_hits_new2.head(50)

,hit_id,x,y,z,volume_id,layer_id,module_id,particle_id,tx,ty,...,py,pz,q,nhits,x2,y2,z2,labels,labels1,num_label_instances
74006,74007,-235.235001,-438.778015,-97.599998,13,6,716,4505729947934722,-235.246002,-438.769989,...,-0.183371,-0.004429,1.0,9.0,-0.463669,-0.864871,-0.196040,5482,522606,2
53830,53831,-359.197998,408.306000,-2954.500000,12,2,148,4505867370102784,-358.955994,408.057007,...,0.198164,-2.231560,1.0,13.0,-0.119568,0.135915,-5.432893,6257,583819,4
66856,66857,353.069000,80.057503,-543.799988,13,4,310,4507104320684032,353.067993,80.066902,...,0.378889,-1.882020,1.0,11.0,0.540449,0.122545,-1.502079,736,68002,2
19232,19233,12.092300,-32.017700,74.391098,8,2,134,4507241759637504,12.092900,-32.017101,...,-0.296460,0.614653,1.0,4.0,0.147672,-0.391001,2.173584,5761,546412,4
19205,19206,11.355800,-29.859699,70.021797,8,2,133,4507241759637504,11.357300,-29.859200,...,-0.296460,0.614653,1.0,4.0,0.147545,-0.387965,2.191870,5761,546412,4
17616,17617,11.190600,-29.393000,69.062599,8,2,117,4507241759637504,11.195400,-29.391300,...,-0.296460,0.614653,1.0,4.0,0.147464,-0.387326,2.195865,5761,546412,4
102101,102102,690.797974,-353.709015,1497.500000,18,4,42,4508547446484994,695.315979,-355.946991,...,0.198347,0.251084,1.0,7.0,0.409566,-0.209710,1.929550,500,45801,4
14332,14333,31.874201,12.252900,-168.947998,8,2,74,4515488096845824,31.873400,12.253900,...,0.718661,-10.555200,-1.0,13.0,0.184923,0.071087,-4.947496,3030,292201,2
14298,14299,29.824301,11.457100,-157.498993,8,2,73,4515488096845824,29.824900,11.455300,...,0.718661,-10.555200,-1.0,13.0,0.185582,0.071292,-4.929663,3030,292202,5
21629,21630,68.122398,23.163401,-350.141998,8,4,82,4515762974752768,68.122597,23.162901,...,0.589758,-9.466710,-1.0,5.0,0.190574,0.064800,-4.866274,3030,292202,5


In [11]:
i

6656

In [13]:
X_test = all_hits_new2[['x2', 'y2', 'z2']].values

p = model.predict(X_test.reshape(X_test.shape[0],1,3))    
p = [np.argmax(p[j]) + 1 + (i * 100) for j in range(len(p))]


all_hits_new2['labels2'] = p

In [14]:
all_hits_new2.head(50)

,hit_id,x,y,z,volume_id,layer_id,module_id,particle_id,tx,ty,...,pz,q,nhits,x2,y2,z2,labels,labels1,num_label_instances,labels2
74006,74007,-235.235001,-438.778015,-97.599998,13,6,716,4505729947934722,-235.246002,-438.769989,...,-0.004429,1.0,9.0,-0.463669,-0.864871,-0.196040,5482,522606,2,665606
53830,53831,-359.197998,408.306000,-2954.500000,12,2,148,4505867370102784,-358.955994,408.057007,...,-2.231560,1.0,13.0,-0.119568,0.135915,-5.432893,6257,583819,4,665619
66856,66857,353.069000,80.057503,-543.799988,13,4,310,4507104320684032,353.067993,80.066902,...,-1.882020,1.0,11.0,0.540449,0.122545,-1.502079,736,68002,2,665602
19232,19233,12.092300,-32.017700,74.391098,8,2,134,4507241759637504,12.092900,-32.017101,...,0.614653,1.0,4.0,0.147672,-0.391001,2.173584,5761,546412,4,665612
19205,19206,11.355800,-29.859699,70.021797,8,2,133,4507241759637504,11.357300,-29.859200,...,0.614653,1.0,4.0,0.147545,-0.387965,2.191870,5761,546412,4,665612
17616,17617,11.190600,-29.393000,69.062599,8,2,117,4507241759637504,11.195400,-29.391300,...,0.614653,1.0,4.0,0.147464,-0.387326,2.195865,5761,546412,4,665612
102101,102102,690.797974,-353.709015,1497.500000,18,4,42,4508547446484994,695.315979,-355.946991,...,0.251084,1.0,7.0,0.409566,-0.209710,1.929550,500,45801,4,665601
14332,14333,31.874201,12.252900,-168.947998,8,2,74,4515488096845824,31.873400,12.253900,...,-10.555200,-1.0,13.0,0.184923,0.071087,-4.947496,3030,292201,2,665601
14298,14299,29.824301,11.457100,-157.498993,8,2,73,4515488096845824,29.824900,11.455300,...,-10.555200,-1.0,13.0,0.185582,0.071292,-4.929663,3030,292202,5,665602
21629,21630,68.122398,23.163401,-350.141998,8,4,82,4515762974752768,68.122597,23.162901,...,-9.466710,-1.0,5.0,0.190574,0.064800,-4.866274,3030,292202,5,665602


In [16]:
one_submission = create_one_event_submission(0, all_hits_new2, all_hits_new2['labels2'].values)
#     dataset_submissions.append(one_submission)

# Score for the event
score = score_event(all_hits_new2, one_submission)
print(score)

0.0


In [6]:
model_100 = load_model('../cache/checkpoint/checkpoint-6488-100-1-3000.hdf5')

In [22]:
X_test = all_hits_new2[['x2', 'y2', 'z2']].values

p = model_100.predict(X_test.reshape(X_test.shape[0],1,3))    
p = [np.argmax(p[j]) + 1 + (i * 10000) for j in range(len(p))]


all_hits_new2['labels3'] = p

one_submission = create_one_event_submission(0, all_hits_new2, all_hits_new2['labels3'].values)
#     dataset_submissions.append(one_submission)

# Score for the event
score = score_event(all_hits_new2, one_submission)
print(score)

0.008186267388620426


In [23]:
all_hits_new2.head(50)

,hit_id,x,y,z,volume_id,layer_id,module_id,particle_id,tx,ty,...,q,nhits,x2,y2,z2,labels,labels1,num_label_instances,labels2,labels3
74006,74007,-235.235001,-438.778015,-97.599998,13,6,716,4505729947934722,-235.246002,-438.769989,...,1.0,9.0,-0.463669,-0.864871,-0.196040,5482,522606,2,665606,66560030
53830,53831,-359.197998,408.306000,-2954.500000,12,2,148,4505867370102784,-358.955994,408.057007,...,1.0,13.0,-0.119568,0.135915,-5.432893,6257,583819,4,665619,66560093
66856,66857,353.069000,80.057503,-543.799988,13,4,310,4507104320684032,353.067993,80.066902,...,1.0,11.0,0.540449,0.122545,-1.502079,736,68002,2,665602,66560093
19232,19233,12.092300,-32.017700,74.391098,8,2,134,4507241759637504,12.092900,-32.017101,...,1.0,4.0,0.147672,-0.391001,2.173584,5761,546412,4,665612,66560029
19205,19206,11.355800,-29.859699,70.021797,8,2,133,4507241759637504,11.357300,-29.859200,...,1.0,4.0,0.147545,-0.387965,2.191870,5761,546412,4,665612,66560029
17616,17617,11.190600,-29.393000,69.062599,8,2,117,4507241759637504,11.195400,-29.391300,...,1.0,4.0,0.147464,-0.387326,2.195865,5761,546412,4,665612,66560029
102101,102102,690.797974,-353.709015,1497.500000,18,4,42,4508547446484994,695.315979,-355.946991,...,1.0,7.0,0.409566,-0.209710,1.929550,500,45801,4,665601,66560029
14332,14333,31.874201,12.252900,-168.947998,8,2,74,4515488096845824,31.873400,12.253900,...,-1.0,13.0,0.184923,0.071087,-4.947496,3030,292201,2,665601,66560014
14298,14299,29.824301,11.457100,-157.498993,8,2,73,4515488096845824,29.824900,11.455300,...,-1.0,13.0,0.185582,0.071292,-4.929663,3030,292202,5,665602,66560014
21629,21630,68.122398,23.163401,-350.141998,8,4,82,4515762974752768,68.122597,23.162901,...,-1.0,5.0,0.190574,0.064800,-4.866274,3030,292202,5,665602,66560014


In [7]:
path_to_test = "../data/test"
test_dataset_submissions = []

create_submission = True # True for submission 
delete_file_if_exists('../submissions/tmp-submission-005.csv')
if create_submission:
    for event_id, hits, cells in load_dataset(path_to_test, parts=['hits', 'cells']):

        # Track pattern recognition 
        model1 = Clusterer1()
        labs = model1.predict(hits)
        hits['labels'] = labs
        
        all_hits = pd.DataFrame()
        prev_p_len = 0
        for i, label in tqdm(enumerate(list(set(labs)))):
            myhits = hits[hits['labels'] == label]
            X_test = myhits[['x2', 'y2', 'z2']].values

            p = model_100.predict(X_test.reshape(X_test.shape[0],1,3))    
            p = [np.argmax(p[j]) + 1 + (i * prev_p_len) for j in range(len(p))]
            prev_p_len = len(p)
            if prev_p_len < 100:
                prev_p_len = 100
            myhits['labels1'] = p
            all_hits = all_hits.append(myhits)
        all_hits.sort_values('hit_id', inplace=True)
    
        # Prepare submission for an event
#         one_submission = create_one_event_submission(event_id, hits, labels)
        one_submission = create_one_event_submission(event_id, all_hits, all_hits['labels1'].values)
        test_dataset_submissions.append(one_submission)
        append_to_csv(one_submission, "../submissions/tmp-submission-005.csv")
        print('Event ID: ', event_id)

    # Create submission file
    submission = pd.concat(test_dataset_submissions, axis=0)
    submission.to_csv('../submissions/submission-005.csv', index=False)

100%|██████████| 9157/9157 [00:00<00:00, 154282.51it/s]
9156it [18:48,  8.11it/s]


Event ID:  0


100%|██████████| 9453/9453 [00:00<00:00, 228484.90it/s]
9453it [19:20,  8.15it/s]


Event ID:  1


100%|██████████| 8113/8113 [00:00<00:00, 158400.50it/s]
8112it [16:35,  8.15it/s]


...

100%|██████████| 7885/7885 [00:00<00:00, 261179.28it/s]
7885it [15:08,  8.68it/s]


Event ID:  123


100%|██████████| 8288/8288 [00:00<00:00, 260663.85it/s]
8288it [15:59,  8.64it/s]


Event ID:  124


FileNotFoundError: [Errno 2] No such file or directory: 'submissions/submission-005.csv'

In [8]:
submission.to_csv('../submissions/submission-005.csv', index=False)